In [1]:
import numpy as np
import scipy as scipy
import skfuzzy as fuzz
import pandas as pd
import sklearn.metrics as Mestric
from sklearn.decomposition import NMF
import networkx as nx


In [2]:
dataset = nx.karate_club_graph()
G = nx.Graph(dataset)
adjacency_matrix = nx.to_numpy_array(G, dtype=int)


In [3]:
def run_nmf(matrix, n_components):
    model = NMF(n_components=n_components, init='random', random_state=0)
    w = model.fit_transform(matrix)
    h = model.components_
    return model,w,h

In [4]:
def run_fcm(matrix, n_clusters):
    cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(
        matrix, n_clusters, 2, error=0.005, maxiter=1000)
    return cntr, u

In [5]:
#NMF
n_components = 2
model,W,H = run_nmf(adjacency_matrix, n_components)
norms = np.linalg.norm(W.T, ord=2, axis=0)

normalized_W = W.T / norms
W =  normalized_W.T

predicted_labels_nmf = np.argmax(W, axis=1)


In [6]:
#FCM 
n_clusters = 2

cntr, u = run_fcm(adjacency_matrix, n_clusters)

predicted_labels_fcm = np.argmax(u, axis=0)

In [7]:
eval =[]

#NMF Metrics
eval.append([
    Mestric.silhouette_score(adjacency_matrix, predicted_labels_nmf) ,
    Mestric.calinski_harabasz_score(adjacency_matrix, predicted_labels_nmf),
    Mestric.davies_bouldin_score(adjacency_matrix, predicted_labels_nmf)
    ]
)

In [8]:
#FCM Metrics
eval.append([
    Mestric.silhouette_score(adjacency_matrix, predicted_labels_fcm) ,
    Mestric.calinski_harabasz_score(adjacency_matrix, predicted_labels_fcm),
    Mestric.davies_bouldin_score(adjacency_matrix,predicted_labels_fcm)
    ]
)

In [9]:
titles = ['Silhouette Score' , 'Calinski-Harabasz Index', 'Davies-Bouldin Index']
df_evaluation = pd.DataFrame(eval, columns=titles)
df_evaluation.index = ['NMF', 'FCM']

df_evaluation

,Silhouette Score,Calinski-Harabasz Index,Davies-Bouldin Index
NMF,0.142958,6.383212,2.050306
FCM,0.142958,6.383212,2.050306
